In [1]:
from collections import OrderedDict
from typing import List, Tuple

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split


import flwr as fl
from flwr.common import Metrics

DEVICE = torch.device("cuda")  # Try "cuda" to train on GPU
print(
    f"Training on {DEVICE} using PyTorch {torch.__version__} and Flower {fl.__version__}"
)

Training on cuda using PyTorch 2.0.1+cu117 and Flower 1.4.0


In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="2,3,4,5,6,7"

In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from src.peft import get_peft_config, get_peft_model, PeftConfig, LoraConfig, PeftModel, TaskType, PrefixTuningConfig

In [4]:
CHECKPOINT = "google/Flan-t5-base"

In [5]:
# peft_config = PrefixTuningConfig(task_type=TaskType.SEQ_2_SEQ_LM, 
#                                  inference_mode=False, 
#                                  prefix_projection = True,
#                                  num_virtual_tokens=30)

# model = AutoModelForSeq2SeqLM.from_pretrained(CHECKPOINT)
# model = get_peft_model(model, peft_config)
# model.print_trainable_parameters()
# print(model.prompt_encoder)

In [6]:
# peft_config
# model.prompt_encoder

In [7]:
model = AutoModelForSeq2SeqLM.from_pretrained(CHECKPOINT)

In [8]:
tokenizer = AutoTokenizer.from_pretrained(CHECKPOINT)

In [9]:
from transformers import AdamW
from transformers import logging
from datasets import load_dataset
from evaluate import load as load_metric

In [10]:
from datasets import Dataset
import pandas as pd
#transform pandas type to Dataset for easy implement
# train = Dataset.from_pandas(df_train)
# val = Dataset.from_pandas(df_val)
# test = Dataset.from_pandas(df_test)

In [11]:
from transformers import DataCollatorForSeq2Seq

In [12]:
# for i in range(10):
#     df = pd.read_csv(f"./clients/client_{i}/train.csv")
#     df = df.dropna(subset = ["Input"])
#     df = df.dropna(subset = ["Label"])
#     df.to_csv(f"./clients/client_{i}/train.csv")

In [13]:
# for i in range(10):
#     df = pd.read_csv(f"./clients/client_{i}/test.csv")
#     df = df.dropna(subset = ["Input"])
#     df = df.dropna(subset = ["Label"])
#     df.to_csv(f"./clients/client_{i}/test.csv")

In [14]:
# for i in range(10):
#     df = pd.read_csv(f"./clients/client_{i}/test.csv")
#     df = df[['Input', 'Label', 'assigned']]
#     print(df.columns)
#     df.to_csv(f"./clients/client_{i}/test.csv")


In [15]:
max_input_length = 64
max_target_length = 30
def preprocess_function(examples):
    try :
        model_inputs = tokenizer(
            examples["Input"],
            max_length=max_input_length,
            truncation=True,
        )
        
        labels = tokenizer(
            examples["Label"], 
            max_length=max_target_length, 
            truncation=True
        )
        
        model_inputs["labels"] = labels["input_ids"]
    except :
        pdb.set_trace()
        
    return model_inputs

In [16]:
data_types = {"Input" : str, 'Label' : str}

In [17]:
import pdb

def load_data(clients_num):
    trainloader = []
    testloader = []
    
    dataset_train = [Dataset.from_pandas(pd.read_csv(f"./clients/client_{i}/train.csv",dtype=data_types, usecols = ["Input", "Label"])) for i in range(clients_num)]
    dataset_test = [Dataset.from_pandas(pd.read_csv(f"./clients/client_{i}/test.csv",dtype=data_types, usecols = ["Input", "Label"])) for i in range(clients_num)]
    

    model = AutoModelForSeq2SeqLM.from_pretrained(CHECKPOINT)
    tokenizer = AutoTokenizer.from_pretrained(CHECKPOINT,padding=True,truncation=True)


    # Select 20 random samples to reduce the computation cost
    
#     tokenized_datasets["train"] = tokenized_datasets["train"].select(train_population)
#     tokenized_datasets["test"] = tokenized_datasets["test"].select(test_population)
#     tokenized_datasets = tokenized_datasets.remove_columns("text")
#     tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
#     data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


    tokenized_train = [dataset.map(preprocess_function) for dataset in dataset_train]
    tokenized_test = [dataset.map(preprocess_function) for dataset in dataset_test]
        
    
    data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)    
    for dataset in tokenized_train:
        dataset = dataset.remove_columns(["Input", "Label"])
        
        loader = DataLoader(
            dataset,
            shuffle=True,
            batch_size=8,
            collate_fn=data_collator,
        )
        
        trainloader.append(loader)

        
        
    for dataset in tokenized_test:
        dataset = dataset.remove_columns(["Input", "Label"])
        loader = DataLoader(
            dataset,
            batch_size=8,
            collate_fn=data_collator,
        )
        
        testloader.append(loader)
        

    return trainloader, testloader

In [18]:
# i = 1
# train_set = Dataset.from_pandas(pd.read_csv(f"./clients/client_{i}/train.csv",dtype=data_types, usecols = ["Input", "Label"]))
# train_set = train_set.map(preprocess_function)
# train_set = train_set.remove_columns(["Input", "Label"])

In [19]:
trainloaders, testloaders = load_data(5)

Map:   0%|          | 0/7530 [00:00<?, ? examples/s]

Map:   0%|          | 0/15869 [00:00<?, ? examples/s]

Map:   0%|          | 0/7683 [00:00<?, ? examples/s]

Map:   0%|          | 0/11336 [00:00<?, ? examples/s]

Map:   0%|          | 0/10063 [00:00<?, ? examples/s]

Map:   0%|          | 0/1882 [00:00<?, ? examples/s]

Map:   0%|          | 0/3967 [00:00<?, ? examples/s]

Map:   0%|          | 0/1921 [00:00<?, ? examples/s]

Map:   0%|          | 0/2835 [00:00<?, ? examples/s]

Map:   0%|          | 0/2518 [00:00<?, ? examples/s]

In [21]:
from nltk.tokenize import sent_tokenize

import evaluate
import pdb

def train(net, trainloader, epochs):
    optimizer = AdamW(net.parameters(), lr=5e-5)
    net.train()
    losses = 0
    for _ in range(epochs):
        for batch in trainloader:
            batch = {k: v.to(DEVICE) for k, v in batch.items()}
            outputs = net(**batch)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            losses += loss.item()
        
        print(f"loss : {loss / len(trainloader)}")

def test(net, testloader):
    rouge_score = evaluate.load("rouge")
    loss = 0
    net.eval()
    score_list = []
    for batch in testloader:
        batch = {k: v.to(DEVICE) for k, v in batch.items()}
        labels = batch['labels'].cpu()
        
        with torch.no_grad():
            predictions = net.generate(**batch, max_length = 70)
        
        decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
        labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
        decoded_preds = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
        decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]
        rouge_score.add_batch(predictions=decoded_preds, references=decoded_labels)       
        
        
    # Extract the median ROUGE scores
    result = rouge_score.compute()
    print(decoded_preds)
    print(decoded_labels)
    result = {key: value for key, value in result.items()}
    result = {k: round(v, 4) for k, v in result.items()}
    return result['rouge2'], result['rouge2']

In [22]:
# train(model.to(DEVICE), trainloaders[0], 1)
res_list = []
for i in range(5):
    res = test(model.to(DEVICE), trainloaders[i])
    res_list.append(res)

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


["I'm terribly sorry old man.\nI'm sorry.", "Yeah so?\nThe guy's beat up - he...he probably has a concussion or something, right?\nHe's a snort...he's a snort...he's a snort...he's a sn"]
["I'm terribly sorry old man.", "Yeah so?\nThe guy's beat up - he...he probably has a concussion or something, right?\nHe"]
["That'll be the day... 'As we developed the village'... Next time you develop a village, hit it where the fucking fucking fucking fucking fucking fucking fucking fucking fucking fucking", 'Once they gave me a job to draw up an estimate for the construction of a school.\nIt took me only a day and a half to get the job done.', "Ha Ha!\nYou said, I do.\nI guess that means we're married.\nI'm not sure.", "Yeesh, they sound like a cult.\nBesides, they're greedy bitches.\nThey already have three children.", "Well, I haven't fought just one person for so long.\nI've been specializing in groups.\nBattling a group of people is a way to get them to get out of the way."]
["That'll be the d

In [25]:
ss = 0
for i in res_list:
    ss += i[0]

ss/5

0.7130600000000001

In [19]:
class IMDBClient(fl.client.NumPyClient):
    def __init__(self, net, trainloader, testloader):
        self.net = net
        self.trainloader = trainloader
        self.testloader = testloader

    def get_parameters(self, config):
        return [val.cpu().numpy() for _, val in self.net.state_dict().items()]

    def set_parameters(self, parameters):
        params_dict = zip(self.net.state_dict().keys(), parameters)
        state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
        self.net.load_state_dict(state_dict, strict=True)

    def fit(self, parameters, config):
        self.set_parameters(parameters)
        print("Training Started...")
        train(self.net, self.trainloader, epochs=1)
        print("Training Finished.")
        return self.get_parameters(config={}), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        self.set_parameters(parameters)
        loss, accuracy = test(self.net, self.testloader)
        return float(loss), len(self.testloader), {"accuracy": float(accuracy), "loss": float(loss)}

In [20]:
class PrefixClient(fl.client.NumPyClient):
    def __init__(self, net, trainloader, testloader):
        self.net = net
        self.trainloader = trainloader
        self.testloader = testloader

    def get_parameters(self, config):
        return self.net.get_prompt_parameters()

    def set_parameters(self, parameters):
        self.net.set_prompt_parameters(parameters)

    def fit(self, parameters, config):
        self.set_parameters(parameters)
        print("Training Started...")
        train(self.net, self.trainloader, epochs=1)
        print("Training Finished.")
        return self.get_parameters(config={}), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        self.set_parameters(parameters)
        loss, accuracy = test(self.net, self.testloader)
        return float(loss), len(self.testloader), {"accuracy": float(accuracy), "loss": float(loss)}

In [21]:
def client_fn(cid: str) -> IMDBClient:
    """Create a Flower client representing a single organization."""

    # Load model
    net = model.to(DEVICE)

    # Load data (CIFAR-10)
    # Note: each client gets a different trainloader/valloader, so each client
    # will train and evaluate on their own unique data
    trainloader = trainloaders[int(cid)]
    valloader = testloaders[int(cid)]

    # Create a  single Flower client representing a single organization
    return IMDBClient(net, trainloader, valloader)

In [ ]:
# Create FedAvg strategy
strategy = fl.server.strategy.FedAvg(
    fraction_fit=1.0,  # Sample 100% of available clients for training
    fraction_evaluate=1.0,  # Sample 50% of available clients for evaluation
    min_fit_clients=5,  # Never sample less than 10 clients for training
    min_evaluate_clients=5,  # Never sample less than 5 clients for evaluation
    min_available_clients=5,  # Wait until all 10 clients are available
)

# Specify client resources if you need GPU (defaults to 1 CPU and 0 GPU)
client_resources = None
if DEVICE.type == "cuda":
    client_resources = {"num_gpus": 1}

NUM_CLIENTS = 5
# Start simulation
history = fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=10),
    strategy=strategy,
    client_resources=client_resources,
#     ray_init_args={"include_dashboard": False}
)

INFO flwr 2023-08-05 13:49:00,477 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

2023-08-05 13:49:03,282	INFO worker.py:1621 -- Started a local Ray instance.
INFO flwr 2023-08-05 13:49:04,706 | app.py:180 | Flower VCE: Ray initialized with resources: {'object_store_memory': 29529923174.0, 'GPU': 6.0, 'memory': 59059846350.0, 'accelerator_type:RTX': 1.0, 'CPU': 40.0, 'node:__internal_head__': 1.0, 'node:202.30.1.208': 1.0}
INFO flwr 2023-08-05 13:49:04,708 | server.py:86 | Initializing global parameters
INFO flwr 2023-08-05 13:49:04,710 | server.py:273 | Requesting initial parameters from one random client
INFO flwr 2023-08-05 13:49:25,151 | server.py:277 | Received initial parameters from one random client
INFO flwr 2023-08-05 13:49:25,154 | server.py:88 | Evaluating initial parameters
INFO flwr 2023-08-05 13:49:25,155 | server.py:101 | FL starting
DEBUG flwr 2023-08-05 13:49:25,157 | server.py:218 | fit_round 1: strategy sampled 5 clients (out of 5)
(launch_and_fit pid=38394) /home/qudgks/miniconda3/envs/FL/lib/python3.8/site-packages/transformers/optimization.py:

(launch_and_fit pid=38394) Training Started...
(launch_and_fit pid=38481) loss : 0.0036469155456870794
(launch_and_fit pid=38481) Training Finished.
(launch_and_fit pid=38496) Training Started... [repeated 4x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)
(launch_and_fit pid=38437) loss : 0.003442393383011222
(launch_and_fit pid=38437) Training Finished.
(launch_and_fit pid=38496) loss : 0.002363845007494092
(launch_and_fit pid=38496) Training Finished.
(launch_and_fit pid=38394) loss : 0.0023197701666504145
(launch_and_fit pid=38394) Training Finished.
(launch_and_fit pid=38577) loss : 0.001842861413024366
(launch_and_fit pid=38577) Training Finished.


DEBUG flwr 2023-08-05 13:56:59,259 | server.py:232 | fit_round 1 received 5 results and 0 failures
WARNING flwr 2023-08-05 13:57:23,265 | fedavg.py:243 | No fit_metrics_aggregation_fn provided
DEBUG flwr 2023-08-05 13:57:23,298 | server.py:168 | evaluate_round 1: strategy sampled 5 clients (out of 5)
(launch_and_fit pid=38496) /home/qudgks/miniconda3/envs/FL/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning [repeated 4x across cluster]
(launch_and_fit pid=38496)   warnings.warn( [repeated 4x across cluster]
(launch_and_evaluate pid=3857) You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding. [r

...

DEBUG flwr 2023-08-05 14:53:19,999 | server.py:232 | fit_round 5 received 4 results and 1 failures
DEBUG flwr 2023-08-05 14:53:31,438 | server.py:168 | evaluate_round 5: strategy sampled 5 clients (out of 5)
(launch_and_evaluate pid=7138) You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding. [repeated 5x across cluster]
(launch_and_fit pid=703) /home/qudgks/miniconda3/envs/FL/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning [repeated 3x across cluster]
(launch_and_fit pid=703)   warnings.warn( [repeated 3x across cluster]


(launch_and_evaluate pid=7171) ["Funnier to who and at who's expense?\nDunwitty, when Negroes start to run amok, the boycotts", "Yes, with your clients' names attached.\nThat's the only reason those poor slobs pay you - to see their names", "Dr. Bloom showed me your article on surgical addiction in the journal of Clinical Psychiatry.\nI'm sorry, I'm sorry.", "It's the truth.\nThose 17 superstars are our insurance policy.\nWe can't open -- can't make a profit --", "Losing power.\nThe laser drained it.\nI'm not sure.", "You mock us with a smile?\nI'm not a snob.", "I've got my Dr. Gruber, says her heart condition means they gave her the wrong anaesthetic anyway, plus she's"]
(launch_and_evaluate pid=7171) ["Funnier to who and at who's expense?\nDunwitty, when Negroes start to run amok, the boycotts", "Yes, with your clients' names attached.\nThat's the only reason those poor slobs pay you - to see their names", 'Dr. Bloom showed me your article on surgical addiction in the journal of Cli

DEBUG flwr 2023-08-05 15:00:39,118 | server.py:182 | evaluate_round 5 received 5 results and 0 failures
DEBUG flwr 2023-08-05 15:00:39,120 | server.py:218 | fit_round 6: strategy sampled 5 clients (out of 5)


(launch_and_evaluate pid=7138) ["... That's funny...\nI had this job with Enzo... we got fired too.", "Five and a half minutes.\nHe's breached the hot zone..\nI'm not going to be able to get him.", "No.\nNever heard of him.\nI'm not sure.", "A what?\nI'm a paleontologist.", "What was the color of Tim's belt?\nI'm not sure.\nI'm not sure.\nI'm not sure.\nI'm not sure.\nI'm not sure.\nI'm not sure.\nI'm not sure.\nI'm not sure.\nI", "No.\nHeadin' west.\nTo Can-tuck-ee.\nI'm not going to bet on it."]
(launch_and_evaluate pid=7138) ["... That's funny...\nI had this job with Enzo... we got fired too.", "Five and a half minutes.\nHe's breached the hot zone..", 'No.\nNever heard of him.', 'A what?', "What was the color of Tim's belt?", "No.\nHeadin' west.\nTo Can-tuck-ee."]
(launch_and_fit pid=12386) Training Started...


(launch_and_fit pid=12386) You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding. [repeated 5x across cluster]
(launch_and_fit pid=12386) /home/qudgks/miniconda3/envs/FL/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
(launch_and_fit pid=12386)   warnings.warn(
(launch_and_fit pid=12441) /home/qudgks/miniconda3/envs/FL/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warnin

In [24]:
history

History (loss, distributed):
	round 1: 0.710209068776628
	round 2: 0.6990351186853316
	round 3: 0.6785576384662204
	round 4: 0.6928872793670116
	round 5: 0.6753511868533172
	round 6: 0.6980922253922968
	round 7: 0.6756048691418136
	round 8: 0.6608476708074534
	round 9: 0.7113708460133902
	round 10: 0.6638261107729764

In [ ]:
#base line
History (loss, distributed):
	round 1: 0.710209068776628
	round 2: 0.6990351186853316
	round 3: 0.6785576384662204
	round 4: 0.6928872793670116
	round 5: 0.6753511868533172
	round 6: 0.6980922253922968
	round 7: 0.6756048691418136
	round 8: 0.6608476708074534
	round 9: 0.7113708460133902
	round 10: 0.6638261107729764

In [ ]:
# prefix round 10
History (loss, distributed):
	round 1: 0.7222987827145466
	round 2: 0.7267468654899574
	round 3: 0.7276815581253804
	round 4: 0.7292324406573341
	round 5: 0.7272822276323798
	round 6: 0.729241387705417
	round 7: 0.7286804625684723
	round 8: 0.7295257455873402
	round 9: 0.7313477175897748
	round 10: 0.7306583688374925

In [ ]:
#round 2 prefix
History (loss, distributed):
	round 1: 0.7422124581175754
	round 2: 0.7481604629911666
# round 2 baseline
History (loss, distributed):
	round 1: 0.7218715199512641
	round 2: 0.7248902223575997

In [ ]:
# def weighted_average(metrics):
#     accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
#     losses = [num_examples * m["loss"] for num_examples, m in metrics]
#     examples = [num_examples for num_examples, _ in metrics]
#     return {"accuracy": sum(accuracies) / sum(examples), "loss": sum(losses) / sum(examples)}

# strategy = fl.server.strategy.FedAvg(
#     fraction_fit=1.0,
#     fraction_evaluate=1.0,
#     evaluate_metrics_aggregation_fn=weighted_average,
# )

# fl.simulation.start_simulation(
#     client_fn=client_fn,
#     num_clients=NUM_CLIENTS,
#     config=fl.server.ServerConfig(num_rounds=NUM_ROUNDS),
#     strategy=strategy,
#     client_resources={"num_cpus": 1, "num_gpus": 0},
#     ray_init_args={"log_to_driver": False, "num_cpus": 1, "num_gpus": 0}
# )

In [ ]:
hostory